In [1]:
import os
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from datetime import datetime


In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

## Функция для вариации датасета

In [3]:
# Вспомогательные функции
blist = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90+']

# Функция назначения номера возрастной группы по значению возраста
def binagex(a):
    x = a // 10
    x = 9 if x > 9 else x
    return x

# утилита
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)


In [4]:
### Добавление названий имен знаков зодиака

# Загрузка пакета и функции генерации
!pip install zodiac-sign

from zodiac_sign import get_zodiac_sign

In [5]:
def zsign(d):   # тест неверного паттерна - ошибки при выполнении
    return get_zodiac_sign(d['birthday'])

# знак зодиака на русском
def zodiac_name(row):
    return get_zodiac_sign(row['birthdate'], language='ru_RU')

def zodiac_local(locale=None):
    _n = locale if locale else ''
    def z_(row):
      if locale:
        return get_zodiac_sign(row['birthdate'], language=locale)
      else:
        return get_zodiac_sign(row['birthdate'])
    z_.__name__ = 'zodiac'+_n
    return z_


In [32]:
def DS_Gen(ds, delcols=[], **kwargs):
    '''
    Функция для аугментации датасета
    параметры:
        ds: начальный pd.resFrame
        delcols: список имен колонок, которые надо удалить из датасета
        необязательные именованные аргументы
            1. имя_колонки=True - Если в функции встроен генератор колонки с таким именем, то он будет выполняться
            2. creators=[список внешних функций, которые будут выполнены]
               creator - внешняя функция с одним входным аргументом: строкой датасета.
                         Вызывается через data.apply(creator,axis=1).
                         Может обращаться к любым колонкам текушей записи
            3. get_OHE = [список колонок для развертывания в OHE векторы]
    Возвращает: pd.resFrame

    Пример:
            # Удаление трех колонок, добавление групп по возрвстам (OHE)
            data = DS_Gen(data, delcols=['institution_id','diagnos', 'sss'], agebin=True)
            # Без удаления, добавить колонки смещения критического возраста
            data = DS_Gen(data, critical_age=True)
            # выполнить внешние функции (добавление знака зодиака)
            Creators = [zsign, zodiac_name]
            data = DS_Gen(data, critical_age=True, creators=Creators)
    '''
    res = ds.copy()      #чтение исходного датасета

    # Обработка встроенных колонок
    # ---------------------------------
    # Сейчас реализованы:
    #             critical_age, agebin

    # Константы
    TR_AGE = 40  # принятый порог критического возраста
    OHE = True   # Развертывать ли agebin в OHE колонки

    def _critical_age_(tr_age=TR_AGE):
        if 'age' in res.columns and 'int' in res.age.dtype.name:
            res['critical_age_l']=res['age'].apply(lambda x: tr_age - x if x < tr_age else 0 ).astype('int8')
            res['critical_age_h']=res['age'].apply(lambda x: x - tr_age if x > tr_age else 0  ).astype('int8')

    def _agebin_(ds, ohe=OHE):
        if 'age' in ds.columns and 'int' in ds.age.dtype.name:
            ds['agebix']=ds['age'].apply(lambda x: binagex(x)).astype('int8')
            ds['agebin'] = ds['agebix'].apply(lambda x: blist[x])
            # ds = encode_and_bind(ds, 'agebin') if ohe else ds
            ds = pd.get_dummies(ds, 'agebin') if ohe else ds
        return ds


    ##
    # Добавление встроенных колонок, если признак=True в kwargs
    #
    if 'critical_age' in kwargs and kwargs['critical_age']:
        _critical_age_()
        print(f"добавлены 'critical_age_l', 'critical_age_h'")

    if 'agebin' in kwargs and  kwargs['agebin']:
        res = _agebin_(res,True)
        print(f"добавлены 'agebix' и agebin_OHE векторы")

    if 'get_OHE' in kwargs :
        # _lst_ = kwargs['get_OHE']
        _colist = []
        for c in list(kwargs['get_OHE']):    #_lst_:
          if c in res.columns:
            _colist.append(c)
          else:
            print( f'{c} не найдена в исходном датасете')
            
        res = pd.get_dummies(res, columns=_colist, drop_first=True)
        print(f'добавлены OHE колонки: {_colist}')


    ##
    # Запуск внешних креаторов колонок, если передан список функций
    #
    if 'creators' in kwargs:      # and len(kwargs['creators']) > 0:
        creators = kwargs['creators']
        for c in creators:
          try:
            res[c.__name__] = res.apply(c, axis=1)
            print(f'Выполнена функция {c.__name__}')
          except:
            print(f'ошибка при выполнении функции {c.__name__}')



    # Удаление колонок по списку
    #
    _dc = []
    for c in delcols:
        if c in res.columns:
            _dc.append(c)
        else:
          #  raise f'{c} не найдена в исходном датасете'
          print( f'{c} не найдена в исходном датасете')
    res.drop(columns=_dc, axis=1, inplace=True )
    print(f'Удалены: {_dc}')


    # завершение работы функции
    return res

## Тесты

In [ ]:
# из библиотеки google.colab импортируем класс files
from google.colab import files

# создаем объект этого класса, применяем метод .upload()
uploaded = files.upload()

Saving dataset_wide_birthday.xlsx to dataset_wide_birthday.xlsx


In [7]:
# Рабочий каталог и имя рабочего файла датасета
catalogs_path = ''
wrk_dataset_filename = 'dataset_wide_birthday.xlsx'

In [20]:
# Загрузка датасета со всеми столбцами из текущей папки
data = pd.read_excel(catalogs_path + wrk_dataset_filename)
backup = data.copy()

In [16]:
data.columns

Index(['institution', 'id', 'gender', 'age', 'dose', 'date_analyse',
       'prev_INR(MHO)', 'atrial_fibrillation', 'mitral_valve', 'aortal_valve',
       'date_diff', 'INR(MHO)', 'sample', 'pass_dose', 'institution_id',
       'diagnos', 'sss', 'date_from_1990', 'ageix', 'agebin_0-9',
       'agebin_10-19', 'agebin_20-29', 'agebin_30-39', 'agebin_40-49',
       'agebin_50-59', 'agebin_60-69', 'agebin_70-79', 'agebin_80-89',
       'agebin_90+', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8',
       'deviation_from_median', 'age_percentile', 'age_quantile', 'is_elderly',
       'age_density', 'age_zscore', 'birthdate'],
      dtype='object')

In [10]:
# тестовый список колонок для удаления
dellst = [ 'agebin_0-9',
       'agebin_10-19', 'agebin_20-29', 'agebin_30-39', 'agebin_40-49',
       'agebin_50-59', 'agebin_60-69', 'agebin_70-79', 'agebin_80-89',
       'agebin_90+','critical_age_l','critical_age_h' ]    #'agebix',



In [11]:
data = DS_Gen(data)
display(data.head())

Удалены: []


,institution,id,gender,age,dose,date_analyse,prev_INR(MHO),atrial_fibrillation,mitral_valve,aortal_valve,...,d6,d7,d8,deviation_from_median,age_percentile,age_quantile,is_elderly,age_density,age_zscore,birthdate
0,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-11,3.11,1,0,0,...,0,0,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11
1,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-24,3.32,1,0,0,...,0,0,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11
2,ГБУЗ РБ Толбазинская ЦРБ,140361892057264,1,50,5.00,2022-11-22,2.01,0,1,0,...,0,1,0,-17,5.502392,1,0,0.007255,-1.606331,1972-11-22
3,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-11-21,1.42,1,0,0,...,0,0,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21
4,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-12-06,2.56,1,0,0,...,0,0,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21


In [17]:
data = DS_Gen(data, delcols=dellst)
display(data.head())

critical_age_l не найдена в исходном датасете
critical_age_h не найдена в исходном датасете
Удалены: ['agebin_0-9', 'agebin_10-19', 'agebin_20-29', 'agebin_30-39', 'agebin_40-49', 'agebin_50-59', 'agebin_60-69', 'agebin_70-79', 'agebin_80-89', 'agebin_90+']


,institution,id,gender,age,dose,date_analyse,prev_INR(MHO),atrial_fibrillation,mitral_valve,aortal_valve,...,d6,d7,d8,deviation_from_median,age_percentile,age_quantile,is_elderly,age_density,age_zscore,birthdate
0,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-11,3.11,1,0,0,...,0,0,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11
1,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-24,3.32,1,0,0,...,0,0,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11
2,ГБУЗ РБ Толбазинская ЦРБ,140361892057264,1,50,5.00,2022-11-22,2.01,0,1,0,...,0,1,0,-17,5.502392,1,0,0.007255,-1.606331,1972-11-22
3,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-11-21,1.42,1,0,0,...,0,0,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21
4,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-12-06,2.56,1,0,0,...,0,0,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21


In [13]:
data = DS_Gen(data, delcols=dellst, agebin=True)
display(data.head())

добавлены 'agebix' и agebin_OHE векторы
agebin_0-9 не найдена в исходном датасете
critical_age_l не найдена в исходном датасете
critical_age_h не найдена в исходном датасете
Удалены: ['agebin_10-19', 'agebin_20-29', 'agebin_30-39', 'agebin_40-49', 'agebin_50-59', 'agebin_60-69', 'agebin_70-79', 'agebin_80-89', 'agebin_90+']


,id,gender,age,dose,date_analyse,prev_INR(MHO),atrial_fibrillation,mitral_valve,aortal_valve,date_diff,...,agebin_онмк 08.07.22,agebin_онмк 2008,agebin_онмк 2021,agebin_пикс 2016,"agebin_плоскостная резекция створок аортального клапана,эндартэрэктомия из ПКА, ПМЖВ, аортокоронарошунтирлование ПКА, ВТК; МКШ от 2017 года,",agebin_протезирование АоК 2018,"agebin_протезирование митрального клапана, пластика трикуспидального клапана от 2008 года, МИК 1990 г.",agebin_стенокардия,agebin_стентирование,"agebin_стентирование , АКШ, МКШ"
0,140361892057136,0,57,7.50,2022-11-11,3.11,1,0,0,13,...,0,0,0,0,0,0,0,0,0,0
1,140361892057136,0,57,7.50,2022-11-24,3.32,1,0,0,15,...,0,0,0,0,0,0,0,0,0,0
2,140361892057264,1,50,5.00,2022-11-22,2.01,0,1,0,26,...,0,0,0,0,0,0,0,0,0,0
3,140361892057392,0,53,6.25,2022-11-21,1.42,1,0,0,15,...,0,0,0,0,0,0,0,0,0,0
4,140361892057392,0,53,6.25,2022-12-06,2.56,1,0,0,49,...,0,0,0,0,0,0,0,0,0,0


In [42]:
data = backup.copy()
# 'institution', 'id', 'gender', 'age', 'dose', 'ageix',
data = DS_Gen(data, delcols=['date_analyse',
       'prev_INR(MHO)', 'atrial_fibrillation', 'mitral_valve', 'aortal_valve',
       'date_diff', 'INR(MHO)', 'sample', 'pass_dose', 'institution_id',
       'diagnos', 'sss', 'date_from_1990',  'agebin_0-9',
       'agebin_10-19', 'agebin_20-29', 'agebin_30-39', 'agebin_40-49',
       'agebin_50-59', 'agebin_60-69', 'agebin_70-79', 'agebin_80-89',
       'agebin_90+', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8',
       'deviation_from_median', 'age_percentile', 'age_quantile', 'is_elderly',
       'age_density', 'age_zscore', 'birthdate']
       )

Удалены: ['date_analyse', 'prev_INR(MHO)', 'atrial_fibrillation', 'mitral_valve', 'aortal_valve', 'date_diff', 'INR(MHO)', 'sample', 'pass_dose', 'institution_id', 'diagnos', 'sss', 'date_from_1990', 'agebin_0-9', 'agebin_10-19', 'agebin_20-29', 'agebin_30-39', 'agebin_40-49', 'agebin_50-59', 'agebin_60-69', 'agebin_70-79', 'agebin_80-89', 'agebin_90+', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'deviation_from_median', 'age_percentile', 'age_quantile', 'is_elderly', 'age_density', 'age_zscore', 'birthdate']


In [34]:
data.ageix.unique()

array([5, 6, 7, 8, 4, 3, 2, 9, 1], dtype=int64)

In [41]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)

In [43]:
de = encode_and_bind(data, 'ageix')
de

,institution,id,gender,age,dose
0,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.500
1,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.500
2,ГБУЗ РБ Толбазинская ЦРБ,140361892057264,1,50,5.000
3,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.250
4,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.250
...,...,...,...,...,...
1458,ГБУЗ РБ Янаульская ЦРБ,139766010360000,1,68,3.750
1459,ГБУЗ РБ Янаульская ЦРБ,139766010360144,1,35,3.750
1460,ГБУЗ РБ Янаульская ЦРБ,139766010360144,1,35,3.750
1461,ГБУЗ РБ Янаульская ЦРБ,139766010360288,0,64,2.500


In [35]:
df = DS_Gen(data[['id','ageix']], get_OHE=['ageix'])
df

добавлены OHE колонки: ['ageix']
Удалены: []


,id,ageix_2,ageix_3,ageix_4,ageix_5,ageix_6,ageix_7,ageix_8,ageix_9
0,140361892057136,0,0,0,1,0,0,0,0
1,140361892057136,0,0,0,1,0,0,0,0
2,140361892057264,0,0,0,1,0,0,0,0
3,140361892057392,0,0,0,1,0,0,0,0
4,140361892057392,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1458,139766010360000,0,0,0,0,1,0,0,0
1459,139766010360144,0,1,0,0,0,0,0,0
1460,139766010360144,0,1,0,0,0,0,0,0
1461,139766010360288,0,0,0,0,1,0,0,0


In [18]:
# Проверка развертывания в ObserveHandler
data = DS_Gen(data, delcols=dellst, get_OHE=['agebix', 'gender', 'institution'])
display(data.head())

agebix не найдена в исходном датасете
добавлены OHE колонки: ['gender', 'institution']
agebin_0-9 не найдена в исходном датасете
agebin_10-19 не найдена в исходном датасете
agebin_20-29 не найдена в исходном датасете
agebin_30-39 не найдена в исходном датасете
agebin_40-49 не найдена в исходном датасете
agebin_50-59 не найдена в исходном датасете
agebin_60-69 не найдена в исходном датасете
agebin_70-79 не найдена в исходном датасете
agebin_80-89 не найдена в исходном датасете
agebin_90+ не найдена в исходном датасете
critical_age_l не найдена в исходном датасете
critical_age_h не найдена в исходном датасете
Удалены: []


,id,age,dose,date_analyse,prev_INR(MHO),atrial_fibrillation,mitral_valve,aortal_valve,date_diff,INR(MHO),...,institution_ГКУЗ РБ РКБ№2,institution_Давлекановская ЦРБ,institution_ЕрмкЦРБ,institution_Зилаирская ЦГБ,institution_ОктЦРБ,institution_Сибайская ЦГБ,institution_ТЦРБ,"institution_ЧУЗ "" РЖД Медицина"" г Стерлитамак",institution_ЧУЗ КБ РЖД-Медицина г.Уфа,institution_Чишминская ЦРБ
0,140361892057136,57,7.50,2022-11-11,3.11,1,0,0,13,3.32,...,0,0,0,0,0,0,0,0,0,0
1,140361892057136,57,7.50,2022-11-24,3.32,1,0,0,15,2.01,...,0,0,0,0,0,0,0,0,0,0
2,140361892057264,50,5.00,2022-11-22,2.01,0,1,0,26,2.75,...,0,0,0,0,0,0,0,0,0,0
3,140361892057392,53,6.25,2022-11-21,1.42,1,0,0,15,2.56,...,0,0,0,0,0,0,0,0,0,0
4,140361892057392,53,6.25,2022-12-06,2.56,1,0,0,49,2.80,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data = DS_Gen(data, delcols=dellst, critical_age=True, agebin=True)
display(data.head())
data.columns

agebin_0-9 не найдена в исходном датасете
critical_age_l не найдена в исходном датасете
critical_age_h не найдена в исходном датасете
Удалены: ['agebix', 'agebin_10-19', 'agebin_20-29', 'agebin_30-39', 'agebin_40-49', 'agebin_50-59', 'agebin_60-69', 'agebin_70-79', 'agebin_80-89', 'agebin_90+']
добавлены 'critical_age_l', 'critical_age_h'
добавлены 'agebix' и agebin_OHE векторы


,institution,id,gender,age,dose,date_analyse,prev_INR(MHO),atrial_fibrillation,mitral_valve,aortal_valve,...,agebix,agebin_10-19,agebin_20-29,agebin_30-39,agebin_40-49,agebin_50-59,agebin_60-69,agebin_70-79,agebin_80-89,agebin_90+
0,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-11,3.11,1,0,0,...,5,0,0,0,0,1,0,0,0,0
1,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-24,3.32,1,0,0,...,5,0,0,0,0,1,0,0,0,0
2,ГБУЗ РБ Толбазинская ЦРБ,140361892057264,1,50,5.00,2022-11-22,2.01,0,1,0,...,5,0,0,0,0,1,0,0,0,0
3,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-11-21,1.42,1,0,0,...,5,0,0,0,0,1,0,0,0,0
4,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-12-06,2.56,1,0,0,...,5,0,0,0,0,1,0,0,0,0


Index(['institution', 'id', 'gender', 'age', 'dose', 'date_analyse',
       'prev_INR(MHO)', 'atrial_fibrillation', 'mitral_valve', 'aortal_valve',
       'date_diff', 'INR(MHO)', 'sample', 'pass_dose', 'institution_id',
       'diagnos', 'sss', 'date_from_1990', 'ageix', 'd1', 'd2', 'd3', 'd4',
       'd5', 'd6', 'd7', 'd8', 'deviation_from_median', 'age_percentile',
       'age_quantile', 'is_elderly', 'age_density', 'age_zscore', 'birthdate',
       'critical_age_l', 'critical_age_h', 'agebix', 'agebin_10-19',
       'agebin_20-29', 'agebin_30-39', 'agebin_40-49', 'agebin_50-59',
       'agebin_60-69', 'agebin_70-79', 'agebin_80-89', 'agebin_90+'],
      dtype='object')

In [ ]:
data = DS_Gen(data, delcols=dellst, critical_age=True, zodiak=True, diff1=True)
display(data.head())
data.columns

agebin_0-9 не найдена в исходном датасете
Удалены: ['agebix', 'agebin_10-19', 'agebin_20-29', 'agebin_30-39', 'agebin_40-49', 'agebin_50-59', 'agebin_60-69', 'agebin_70-79', 'agebin_80-89', 'agebin_90+', 'critical_age_l', 'critical_age_h']
добавлены 'critical_age_l', 'critical_age_h'


,institution,id,gender,age,dose,date_analyse,prev_INR(MHO),atrial_fibrillation,mitral_valve,aortal_valve,...,d8,deviation_from_median,age_percentile,age_quantile,is_elderly,age_density,age_zscore,birthdate,critical_age_l,critical_age_h
0,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-11,3.11,1,0,0,...,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11,0,17
1,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-24,3.32,1,0,0,...,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11,0,17
2,ГБУЗ РБ Толбазинская ЦРБ,140361892057264,1,50,5.00,2022-11-22,2.01,0,1,0,...,0,-17,5.502392,1,0,0.007255,-1.606331,1972-11-22,0,10
3,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-11-21,1.42,1,0,0,...,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21,0,13
4,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-12-06,2.56,1,0,0,...,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21,0,13


Index(['institution', 'id', 'gender', 'age', 'dose', 'date_analyse',
       'prev_INR(MHO)', 'atrial_fibrillation', 'mitral_valve', 'aortal_valve',
       'date_diff', 'INR(MHO)', 'sample', 'pass_dose', 'institution_id',
       'diagnos', 'sss', 'date_from_1990', 'ageix', 'd1', 'd2', 'd3', 'd4',
       'd5', 'd6', 'd7', 'd8', 'deviation_from_median', 'age_percentile',
       'age_quantile', 'is_elderly', 'age_density', 'age_zscore', 'birthdate',
       'critical_age_l', 'critical_age_h'],
      dtype='object')

In [ ]:
# Пустой список внешних функций
Creators = []
data = DS_Gen(data, critical_age=True, creators=Creators)
display(data.head())

Удалены: []
добавлены 'critical_age_l', 'critical_age_h'


,institution,id,gender,age,dose,date_analyse,prev_INR(MHO),atrial_fibrillation,mitral_valve,aortal_valve,...,d8,deviation_from_median,age_percentile,age_quantile,is_elderly,age_density,age_zscore,birthdate,critical_age_l,critical_age_h
0,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-11,3.11,1,0,0,...,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11,0,17
1,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-24,3.32,1,0,0,...,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11,0,17
2,ГБУЗ РБ Толбазинская ЦРБ,140361892057264,1,50,5.00,2022-11-22,2.01,0,1,0,...,0,-17,5.502392,1,0,0.007255,-1.606331,1972-11-22,0,10
3,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-11-21,1.42,1,0,0,...,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21,0,13
4,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-12-06,2.56,1,0,0,...,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21,0,13


In [ ]:
# Внешние функции, знаки зодиака
Creators = [zsign, zodiac_name]
data = DS_Gen(data, critical_age=True, creators=Creators)
display(data.head())

Удалены: []
добавлены 'critical_age_l', 'critical_age_h'
ошибка при выполнении функции zsign
Выполнена функция zodiac_name


,institution,id,gender,age,dose,date_analyse,prev_INR(MHO),atrial_fibrillation,mitral_valve,aortal_valve,...,deviation_from_median,age_percentile,age_quantile,is_elderly,age_density,age_zscore,birthdate,critical_age_l,critical_age_h,zodiac_name
0,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-11,3.11,1,0,0,...,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11,0,17,Скорпион
1,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-24,3.32,1,0,0,...,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11,0,17,Скорпион
2,ГБУЗ РБ Толбазинская ЦРБ,140361892057264,1,50,5.00,2022-11-22,2.01,0,1,0,...,-17,5.502392,1,0,0.007255,-1.606331,1972-11-22,0,10,Стрелец
3,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-11-21,1.42,1,0,0,...,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21,0,13,Скорпион
4,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-12-06,2.56,1,0,0,...,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21,0,13,Скорпион


In [ ]:
# Внешние функции, знаки зодиака
zodiacRu = zodiac_local('ru_RU')
zodiacEn = zodiac_local('en_US')
zodiac0 = zodiac_local()

# Creators = [zsign, zodiac_name, zodiacRu, zodiacEn, zodiac0 ]
data = DS_Gen(data, critical_age=True, creators=[zsign, zodiac_name, zodiacRu, zodiacEn, zodiac0 ])
display(data.head())

Удалены: []
добавлены 'critical_age_l', 'critical_age_h'
ошибка при выполнении функции zsign
Выполнена функция zodiac_name
Выполнена функция zodiacru_RU
Выполнена функция zodiacen_US
Выполнена функция zodiac


,institution,id,gender,age,dose,date_analyse,prev_INR(MHO),atrial_fibrillation,mitral_valve,aortal_valve,...,is_elderly,age_density,age_zscore,birthdate,critical_age_l,critical_age_h,zodiac_name,zodiacru_RU,zodiacen_US,zodiac
0,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-11,3.11,1,0,0,...,0,0.017956,-0.919637,1965-11-11,0,17,Скорпион,Скорпион,Scorpio,Scorpio
1,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-24,3.32,1,0,0,...,0,0.017956,-0.919637,1965-11-11,0,17,Скорпион,Скорпион,Scorpio,Scorpio
2,ГБУЗ РБ Толбазинская ЦРБ,140361892057264,1,50,5.00,2022-11-22,2.01,0,1,0,...,0,0.007255,-1.606331,1972-11-22,0,10,Стрелец,Стрелец,Sagittarius,Sagittarius
3,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-11-21,1.42,1,0,0,...,0,0.010555,-1.312034,1969-11-21,0,13,Скорпион,Скорпион,Scorpio,Scorpio
4,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-12-06,2.56,1,0,0,...,0,0.010555,-1.312034,1969-11-21,0,13,Скорпион,Скорпион,Scorpio,Scorpio
